In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

years = range(2006, 2019)

#LINE CHART POLLUTION
disease = {
    'Cancers' : [
        'di cui tumori maligni della trachea, dei bronchi e dei polmoni',
        'di cui tumori maligni della prostata',
        'di cui tumori maligni del cervello e del sistema nervoso centrale',
        'di cui altri tumori maligni',
        'di cui melanomi maligni della cute',
        "di cui tumori maligni dell'esofago",
        'tumori non maligni (benigni e di comportamento incerto)',
        'di cui leucemia',
        'di cui morbo di hodgkin e linfomi',
        'di cui tumori maligni delle labbra, cavità orale e faringe',
        'di cui tumori maligni della laringe',
        'di cui tumori maligni della tiroide',
        'di cui tumori maligni del seno',
    ],
    'Respiratory' : [
        'polmonite',
        'di cui altre malattie croniche delle basse vie respiratorie',
        'altre malattie del sistema respiratorio',
        'tubercolosi',
        'influenza',
        'di cui asma',
    ],
    'Circulatory' : [
        'du cui altre malattie ischemiche del cuore',
    ],
}
pollutants = {
    'PM10' : 'PM10 (SM2005)',
    'NO2' : 'Biossido di Azoto',
    'PM25' : 'Particelle sospese PM2.5',
    'CO_8h' : 'Monossido di Carbonio',
    'O3' : 'Ozono',
    'SO2' : 'Biossido di Zolfo',
    'C6H6' : 'Benzene'
}
limits = {
    'NO2': 200,
    'PM10': 50,
    'PM25': 25,
    'O3': 120,
    'CO_8h': 10,
    'SO2' : 125,
    'C6H6': 5
}

tot = pd.DataFrame(columns=['Data', 'NomeTipoSensore', 'Valore'])
for year in years:
    df = pd.read_csv(f"../csv/lombardia/air_quality/{str(year)}.csv", encoding='utf-8', sep=',')
    result = df.groupby(by=['NomeTipoSensore'])
    result = result["Valore"].mean().reset_index()
    result.drop(result.index[(result["NomeTipoSensore"] == 'Ammoniaca')], axis=0, inplace=True)
    result.drop(result.index[(result["NomeTipoSensore"] == 'Arsenico')], axis=0, inplace=True)
    result.drop(result.index[(result["NomeTipoSensore"] == 'Benzo(a)pirene')], axis=0, inplace=True)
    result.drop(result.index[(result["NomeTipoSensore"] == 'BlackCarbon')], axis=0, inplace=True)
    result.drop(result.index[(result["NomeTipoSensore"] == 'Cadmio')], axis=0, inplace=True)
    result.drop(result.index[(result["NomeTipoSensore"] == 'Monossido di Azoto')], axis=0, inplace=True)
    result.drop(result.index[(result["NomeTipoSensore"] == 'Nikel')], axis=0, inplace=True)
    result.drop(result.index[(result["NomeTipoSensore"] == 'Ossidi di Azoto')], axis=0, inplace=True)
    result.drop(result.index[(result["NomeTipoSensore"] == 'Piombo')], axis=0, inplace=True)
    result = result.assign(Data = lambda x: year)
    tot = tot.append(result, ignore_index=True)
for x in pollutants:
    tot.loc[tot.NomeTipoSensore == pollutants[x], 'Valore'] = (tot['Valore']/limits[x])*100
final = tot.groupby(by=['Data']).sum().reset_index()

#LINE CHART DEATHS
deaths = pd.read_csv('../csv/lombardia/deaths/deaths_lombardia_from2005.csv', encoding='utf-8', sep=',')
deaths.drop("ITTER107",axis=1,inplace=True)
deaths.drop("SEXISTAT1",axis=1,inplace=True)
deaths.drop("CAUSEMORTE_SL",axis=1,inplace=True)
deaths.drop("Seleziona periodo",axis=1,inplace=True)
deaths.drop("Tipo dato",axis=1,inplace=True)
deaths.drop("Sesso",axis=1,inplace=True)
deaths.drop("Flag Codes",axis=1,inplace=True)
deaths.drop("Flags",axis=1,inplace=True)
deaths.drop(deaths.index[(deaths["TIPO_DATO15"] == "MRATE")],axis=0,inplace=True)

circulatory = deaths[deaths['Causa iniziale di morte - European Short List'].isin(disease['Circulatory'])]
respiratory = deaths[deaths['Causa iniziale di morte - European Short List'].isin(disease['Respiratory'])]
cancers = deaths[deaths['Causa iniziale di morte - European Short List'].isin(disease['Cancers'])]

circulatory = circulatory.groupby(by=['TIME'])
circulatory = circulatory["Value"].sum().reset_index()
respiratory = respiratory.groupby(by=['TIME'])
respiratory = respiratory["Value"].sum().reset_index()
cancers = cancers.groupby(by=['TIME'])
cancers = cancers["Value"].sum().reset_index()
    
#Print figures
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=final['Data'], y=final['Valore'], name="Pollution trend"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=circulatory['TIME'], y=circulatory['Value'], name="Circulatory deseases trend"),
    secondary_y=True,
)
# Add figure title
fig.update_layout(
    title_text="Yearly trend pollution/deaths - Lombardia"
)
# Set x-axis title
fig.update_xaxes(title_text="Years")
# Set y-axes titles
fig.update_yaxes(title_text="<b>Yearly sum</b> pollution level", secondary_y=False)
fig.update_yaxes(title_text="<b>Yearly sum</b> circulatory deseases deaths", secondary_y=True)
fig.show()

#Print figures
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=final['Data'], y=final['Valore'], name="Pollution trend"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=respiratory['TIME'], y=respiratory['Value'], name="Respiratory deseases trend"),
    secondary_y=True,
)
# Add figure title
fig.update_layout(
    title_text="Yearly trend pollution/deaths - Lombardia"
)
# Set x-axis title
fig.update_xaxes(title_text="Years")
# Set y-axes titles
fig.update_yaxes(title_text="<b>Yearly sum</b> pollution level", secondary_y=False)
fig.update_yaxes(title_text="<b>Yearly sum</b> respiratory deseases deaths", secondary_y=True)
fig.show()

#Print figures
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=final['Data'], y=final['Valore'], name="Pollution trend"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=cancers['TIME'], y=cancers['Value'], name="Cancers trend"),
    secondary_y=True,
)
# Add figure title
fig.update_layout(
    title_text="Yearly trend pollution/deaths - Lombardia"
)
# Set x-axis title
fig.update_xaxes(title_text="Years")
# Set y-axes titles
fig.update_yaxes(title_text="<b>Yearly sum</b> pollution level", secondary_y=False)
fig.update_yaxes(title_text="<b>Yearly sum</b> cancer deaths", secondary_y=True)
fig.show()